<table>
    <thead>
        <tr>
            <th colspan="2"><br>
<font size="6" color=orange><div style="text-align:center;"> <span style="border: 3px outset black;">OPENCLASSROOMS</span></div></font></th>
            <p>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><font size = "5"><strong><div style="text-align:center;">
<br>Projet 6
</div></strong>

<font size = "6"><strong><div style="text-align:center;">
<font color=blue> Compétition Kaggle
</div></strong>
    </tbody>
</table>


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\Users\bdelaval\Downloads\train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# <font color=red> Analyse exploratoire </font><a id='partie1.5'> </a>

In [ ]:
data_types_dict = {
                   'time_id': 'int32',
                   'investment_id': 'int16',
                   'target': 'float16',
                  }

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'
    
target = 'target'

In [ ]:
train_df = pd.read_csv(r'C:\Users\bdelaval\Downloads\train.csv', # nrows=5 * 10**4,
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict, 
                       index_col = 0)

In [ ]:
train_df

#### Les features sont déjà standardisées.

## <font color=blue> Target </font><a id='partie1.5'> </a>

In [ ]:
train_df['target'].hist(bins = 100)

#### La distribution semble gaussienne, pas d'outlier. Cela ne devrait pas poser de problème de travailler avec.

In [ ]:
# Traçons également les distributions des cibles de quelques entités aléatoires :

for f in np.random.choice(train_df['investment_id'].unique(), 10):
    train_df[train_df['investment_id'] == f]['target'].hist(bins = 100, alpha = 0.2)

#### Chaque investment_id semble correct aussi.

## <font color=blue> Investment </font><a id='partie1.5'> </a>

In [ ]:
train_df['investment_id'].nunique()

In [ ]:
#train_df['investment_id'].value_counts().plot(kind = 'bar')

#### Nous avons 3579 investissements différents, et la plupart d'entre eux ont une quantité suffisante de points de données, aucun filtrage n'est nécessaire.

## <font color=blue> Features </font><a id='partie1.5'> </a>

#### Avec les 300 features, c'est très compliqué de toutes les observer. Voici quelques distributions graphiques en focus.

In [ ]:
f = 'f_67'
train_df[f].hist(bins = 100)

In [ ]:
f = 'f_109'
train_df[f].hist(bins = 100)

In [ ]:
f = 'f_234'
train_df[f].hist(bins = 100)

In [ ]:
f = 'f_164'
train_df[f].hist(bins = 100)

In [ ]:
train_df[features].nunique().hist()

# <font color=red> Features interaction </font><a id='partie1.5'> </a>

In [ ]:
# analyse d'un échantillon de la population

sample_df = train_df.head(50000)
sample_df.to_csv(r'C:\Users\bdelaval\Downloads\sample_df.csv')
sample_df

In [ ]:
import pandas as pd

sample_df = pd.read_csv(r'C:\Users\bdelaval\Downloads\sample_df.csv')

# ici je m'assure que le time_id dans l'ordre chronologique
sample_df = sample_df.sort_values(by=['time_id'])
sample_df

In [ ]:
data_types_dict = {
                   'time_id': 'int32',
                   'investment_id': 'int16',
                   'target': 'float16',
                  }

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'
    
target = 'target'

In [ ]:
correlation = sample_df[[target] + features].corr()
correlation['target'].iloc[1:].hist(bins = 20)

#### Aucune feature n'est très corrélée avec la cible, vérifions maintenant si il y a des variables très corrélées entre elles.

In [ ]:
import numpy as np

In [ ]:
# Je cherche qu'elles sont les paires corrélées à plus de 0.95

corr_matrix = sample_df.corr().abs()
high_corr_var = np.where(corr_matrix > 0.95)
high_corr_var = [(corr_matrix.columns[x],corr_matrix.columns[y]) for x, y in zip(*high_corr_var) if x != y and x < y]
high_corr_var

#### Regard sur les fortes corrélations:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(sample_df["f_4"], sample_df["f_228"], 'o', alpha = 0.8)
plt.xlabel("f_4")
plt.ylabel("f_228")
plt.show()

import scipy.stats as st
print(st.pearsonr(sample_df["f_4"], sample_df["f_228"])[0])

In [ ]:
plt.plot(sample_df["f_14"], sample_df["f_18"], 'o', alpha = 0.8)
plt.xlabel("f_14")
plt.ylabel("f_18")
plt.show()

print(st.pearsonr(sample_df["f_14"], sample_df["f_18"])[0])

In [ ]:
# Ce code sélectionne les corrélations supérieures à 0.95 et les supprime du df directement

corr_matrix = sample_df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
sample_df.drop(to_drop, axis=1, inplace=True)

In [ ]:
# check s'il y a bien 9 colonnes de moins
sample_df.shape

In [ ]:
import seaborn as sn

plt.figure(figsize = (20, 20))
corrMatrix = sample_df.corr().abs()
sn.heatmap(corrMatrix, annot = False)

# <font color=red> PCA </font><a id='partie1.5'> </a>


#### L'objectif est de réduire le nombre de variables mais en conservant une variance expliquée à 95%.

In [ ]:
X = sample_df.drop(['target','investment_id'], axis=1).values
y = sample_df['target'].values

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.decomposition import PCA

pca_294 = PCA(n_components=294)
pca_294.fit(X)
x_pca_294 = pca_294.transform(X)

In [ ]:
print("Variance explained by all 294 principal components =", 
     sum(pca_294.explained_variance_ratio_* 100))

In [ ]:
pca_294.explained_variance_ratio_* 100

In [ ]:
np.cumsum(pca_294.explained_variance_ratio_* 100)

In [ ]:
fig=plt.figure(figsize=(14,8))
plt.plot(np.cumsum(pca_294.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Explained variance')

In [ ]:
# calcul du nombre de variables dont on a besoin pour expliquer 95% de la variance

pca_95 = PCA(n_components = 0.95)
pca_95.fit(X)
X_pca_95 = pca_95.transform(X)
X_pca_95.shape

In [ ]:
# vérif du résultat précédent

print('Variance explained by the first X principal components =',
     np.cumsum(pca_294.explained_variance_ratio_* 100)[143])

In [ ]:
sample_df.drop(['target','investment_id'], axis=1).var()

In [ ]:
# best correlations variables-target in absolute values

corr_matrix = sample_df.drop(['investment_id'], axis=1).corr()
correl = corr_matrix["target"].sort_values(ascending=False)
correl_abs = correl.abs()

In [ ]:
correl_abs.sort_values(ascending=False)

In [ ]:
# sélection des variables les plus influentes pour le machine learning

df_correl_abs = pd.DataFrame(correl_abs.sort_values(ascending=False))
df_correl_abs_best_variance = df_correl_abs[:-150]
df_correl_abs_best_variance

# <font color=red> Analyse des fortes corrélations </font><a id='partie1.5'> </a>


In [ ]:
plt.plot(sample_df["f_67"], sample_df["target"], 'o', alpha = 0.8)
plt.xlabel("f_67")
plt.ylabel("variable cible")
plt.show()

import scipy.stats as st
print(st.pearsonr(sample_df["f_67"], sample_df["target"])[0])

In [ ]:
plt.plot(sample_df["f_134"], sample_df["target"], 'o', alpha = 0.8)
plt.xlabel("f_134")
plt.ylabel("variable cible")
plt.show()

import scipy.stats as st
print(st.pearsonr(sample_df["f_134"], sample_df["target"])[0])

In [ ]:
plt.plot(sample_df["f_254"], sample_df["target"], 'o', alpha = 0.8)
plt.xlabel("f_254")
plt.ylabel("variable cible")
plt.show()

import scipy.stats as st
print(st.pearsonr(sample_df["f_254"], sample_df["target"])[0])

In [ ]:
plt.plot(sample_df["f_211"], sample_df["target"], 'o', alpha = 0.8)
plt.xlabel("f_211")
plt.ylabel("variable cible")
plt.show()

import scipy.stats as st
print(st.pearsonr(sample_df["f_211"], sample_df["target"])[0])

#### Aucune variable ne montre une belle corrélation avec la cible.

In [ ]:
df_correl_abs_best_variance_transposed = df_correl_abs_best_variance.T
df_correl_abs_best_variance_transposed

In [ ]:
sample_df.head(3)

In [ ]:
sample_df.shape

In [ ]:
# keep only best explained col of the 2 df

df_ml = pd.concat([df_correl_abs_best_variance_transposed[:-1], sample_df], axis=0)
df_ml = df_ml[df_ml.columns[:-151]]
df_ml.shape

In [ ]:
investment = sample_df[['investment_id', 'target']]
investment.shape

In [ ]:
# fetch col investment_id

df_ml2 = pd.concat([investment, df_ml], axis=1)

# <font color=red> Model training </font><a id='partie1.5'> </a>


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X = df_ml.drop(['target'], axis=1)
y = df_ml["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
# data_split_shuffle = False, so that we do not use "future" observations to predict "past" observations

## <font color=blue> Model baseline, linear regression </font><a id='partie1.5'> </a>

#### Pour être en accord avec la métrique de soumission de la compétition Kaggle, je choisis l'indice de Pearson.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from scipy.stats.stats import pearsonr
from sklearn.metrics import make_scorer

def my_scorer(X, y):
    correlation = pearsonr(X, y)[0]
    return correlation

my_func = make_scorer(my_scorer, greater_is_better=True)

In [ ]:
from sklearn.linear_model import LinearRegression

parameters = {}

lr = GridSearchCV(LinearRegression(), 
                  parameters, 
                  scoring=my_func, 
                  cv=5, 
                  n_jobs=-1, 
                  return_train_score=True)

lr.fit(X_train, y_train)

lr.cv_results_

In [ ]:
df_lr_results = pd.DataFrame(lr.cv_results_)
df_lr_results = df_lr_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
df_lr_results

## <font color=blue> ElasticNet </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.linear_model import ElasticNet

parameters = {'alpha': [1e-4, 1e-2, 1, 1e2, 1e4],
              'tol': [1e-6, 1e-4, 1e-2],
              'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}

elasticnet = GridSearchCV(ElasticNet(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1) #n_jobs=-1 pour utiliser tous les coeurs

elasticnet.fit(X_train, y_train)

print(elasticnet.best_params_)

In [ ]:
# save the model to disk

import pickle

filename = 'elasticnet_model.sav'
pickle.dump(elasticnet, open(filename, 'wb'))

In [ ]:
df_en_results = pd.DataFrame(elasticnet.cv_results_)
df_en_results = df_en_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_en = df_en_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_en = best_results_en.head(1)
best_results_en

In [ ]:
# load the model from disk

# elasticnet = pickle.load(open(filename, 'rb'))

## <font color=blue> K Nearest Neighbors </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

parameters = {'n_neighbors':[1, 3, 5, 7],
              'p': [2], # p = 1, is to using manhattan_distance (l1), and euclidean_distance (l2)
              'weights': ['distance'] # si distance:les points proches ont plus d'influence que les points éloignés
             }                       

knn = GridSearchCV(KNeighborsRegressor(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1)

knn.fit(X_train, y_train)

print(knn.best_params_)

In [ ]:
# save the model to disk

filename = 'knn_model.sav'
pickle.dump(knn, open(filename, 'wb'))

In [ ]:
df_knn_results = pd.DataFrame(knn.cv_results_)
df_knn_results = df_knn_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_knn = df_knn_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_knn = best_results_knn.head(1)
best_results_knn

## <font color=blue> Random Forest Regressor </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

parameters = {'max_features': ['auto', 'sqrt', 'log2'],
              'n_estimators': [50,100,300]}    

rf = GridSearchCV(RandomForestRegressor(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1)

rf.fit(X_train, y_train)

print(rf.best_params_)

In [ ]:
# save the model to disk

filename = 'rf_model.sav'
pickle.dump(knn, open(filename, 'wb'))

In [ ]:
df_rf_results = pd.DataFrame(rf.cv_results_)
df_rf_results = df_rf_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_rf = df_rf_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_rf = best_results_rf.head(1)
best_results_rf

#### Gros overfitting. Il faudra jouer sur la profondeur des arbres.

## <font color=blue> Gradient Boosting Regressor </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

parameters = {'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
              'tol': [1e-4, 1e-3, 1e-2],
              'n_estimators': [10, 20]} 

gbr = GridSearchCV(GradientBoostingRegressor(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1)

gbr.fit(X_train, y_train)

print(gbr.best_params_)

In [ ]:
# save the model to disk

filename = 'gb_model.sav'
pickle.dump(knn, open(filename, 'wb'))

In [ ]:
df_gbr_results = pd.DataFrame(gbr.cv_results_)
df_gbr_results = df_gbr_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_gbr = df_gbr_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_gbr = best_results_gbr.head(1)
best_results_gbr

## <font color=blue> Adaboost </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

parameters = {'loss': ['linear', 'square', 'exponential'], 
              'learning_rate': [0.1, 1.0, 10], 
              'n_estimators': [10, 50, 200]}

ab = GridSearchCV(AdaBoostRegressor(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1)

ab.fit(X_train, y_train)

print(ab.best_params_)

In [ ]:
# save the model to disk

filename = 'ad_model.sav'
pickle.dump(knn, open(filename, 'wb'))

In [ ]:
df_ab_results = pd.DataFrame(ab.cv_results_)
df_ab_results = df_ab_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_ab = df_ab_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_ab = best_results_ab.head(1)
best_results_ab

## <font color=blue> LGBM Regressor </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

parameters = {'learning_rate': [0.01, 0.1, 1.0], 
              'n_estimators': [10, 100, 500]}

lgbm = GridSearchCV(AdaBoostRegressor(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1)

lgbm.fit(X_train, y_train)

print(lgbm.best_params_)

In [ ]:
# save the model to disk

filename = 'lgbm_model.sav'
pickle.dump(knn, open(filename, 'wb'))

In [ ]:
df_lgbm_results = pd.DataFrame(lgbm.cv_results_)
df_lgbm_results = df_lgbm_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_lgbm = df_lgbm_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_lgbm = best_results_lgbm.head(1)
best_results_lgbm

## <font color=blue> BayesianRidge </font><a id='partie1.5'> </a>

In [ ]:
from sklearn.linear_model import BayesianRidge

parameters = {'n_iter': [10,300,1000],
              'lambda_1': [1e-8, 1e-6, 1e-3],
              'lambda_2': [1e-8, 1e-6, 1e-3]}

br = GridSearchCV(BayesianRidge(), 
                          parameters, 
                          scoring=my_func, 
                          return_train_score=True,
                          cv=5, 
                          n_jobs=-1)
                  
br.fit(X_train, y_train)

print(br.best_params_)

In [ ]:
# save the model to disk

filename = 'br_model.sav'
pickle.dump(br, open(filename, 'wb'))

In [ ]:
df_br_results = pd.DataFrame(br.cv_results_)
df_br_results = df_br_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_br = df_br_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_br = best_results_br.head(1)
best_results_br

# <font color=red> Visualisation des résultats des modèles sur un graphique </font><a id='partie1.5'> </a>

In [ ]:
df_compare = pd.concat([df_lr_results,best_results_en,best_results_knn,best_results_rf,best_results_gbr,best_results_ab,
                       best_results_lgbm,best_results_br], axis = 0)

model = ['linear_regression','elasticnet','KNeighborsRegressor','random_forest','gradient_boosting','adaboost','lgbm','bayesian_ridge']
df_compare['model'] = model

df_compare

In [ ]:
plt.figure(figsize = (12, 7))
plt.rcParams.update({'font.size': 14})
barWidth = 0.25

y1 = df_compare['mean_train_score']
y2 = df_compare['mean_test_score']
x = df_compare['model']
r1 = range(len(y1))
r2 = [x + barWidth for x in r1]

plt.bar(r1, y1, width = barWidth, color = ['springgreen' for i in y1],
           edgecolor = ['black' for i in y1], linewidth = 1, label = 'mean_train_score')
plt.bar(r2, y2, width = barWidth, color = ['hotpink' for i in y1], 
           edgecolor = ['black' for i in y2], linewidth = 1, label = 'mean_test_score')
plt.xticks([r + barWidth / 2 for r in range(len(y1))], df_compare['model'], rotation = 45)

plt.xlabel("Models")
plt.ylabel("Pearsonr")

plt.title("Coefficient de corrélation avec la target")
plt.legend(bbox_to_anchor = (0.85, 0.9))
plt.grid()

#### On remarque que 2 modèles, KNN et RF font un gros overfitting. Malgré cela, le résultat de Random Forest est le meilleur.

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html


ax = df_compare.plot.bar(x='model', y='mean_test_score', rot=45, figsize=(12, 9))

In [ ]:
feat_importances = pd.Series(test.coef_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
feat_importances = pd.Series(test.coef_, index=X.columns)
feat_importances.nsmallest(10).plot(kind='barh')
plt.show()

## <font color=blue> Model stacking </font><a id='partie1.5'> </a>

#### En récupérant les meilleurs modèles, je construis un modèle empilé.

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('en', ElasticNet()),
    ('gbr', GradientBoostingRegressor())
    ]

reg_stack = StackingRegressor(estimators = estimators,
                        final_estimator = RandomForestRegressor())

params = {'en__alpha': [1e-2],
          'en__tol': [1e-2],
          'en__l1_ratio': [0.3],
          'gbr__alpha': [1e-2],
          'gbr__tol': [1e-2],
          'gbr__n_estimators': [20]}

grid = GridSearchCV(estimator = reg_stack, param_grid = params, cv=5, n_jobs=-1, return_train_score=True)
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
df_grid_results = pd.DataFrame(grid.cv_results_)
df_grid_results = df_grid_results[['mean_fit_time','mean_score_time','mean_train_score','mean_test_score']]
best_results_grid = df_grid_results.sort_values(by=['mean_test_score'], ascending=False)

best_results_grid = best_results_grid.head(1)
best_results_grid

## Le modèle empilé ne fait pas mieux que le meilleur modèle qui est ElasticNet.

In [ ]:
"""
import ubiquant

env = ubiquant.make_env()  
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df.drop(['row_id'], axis=1, inplace=True)
    pred = model.predict(test_df)
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df) 
"""